In [ ]:
# Video Store - Pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Read the input file
vstable = pd.read_csv("/home/roselyne/classes/DSC478/data/video_store.csv", index_col=0)
# Look at the shape of the data matrix
vstable.shape
# 40 objects * 6 features

In [ ]:
# Print first 10 objects
vstable.head(10)

In [ ]:
# Print feature names
vstable.columns

In [ ]:
# What about feature types
vstable.dtypes

#### Now we can convert columns to the appropriate type as necessary:

In [ ]:
vstable["Income"] = vstable["Income"].astype(float)
vstable.dtypes

In [ ]:
# Describe the data
vstable.describe()

In [ ]:
#Alternatively, force describe "all" - we can now see categorical attributes frequencies
vstable.describe(include="all")

In [ ]:
# Select which variables to describe
vstable[["Income", "Age"]].describe()

In [ ]:
# Compute min/max of income
min_sal = vstable["Income"].min()
max_sal = vstable["Income"].max()
print (min_sal, max_sal)

#### We can perform data transformations such as normalization by directly applying the operation to the Pandas Series:

In [ ]:
# Normalize salaries and print first 10
norm_sal = (vstable["Income"] - min_sal) / (max_sal-min_sal)
print (norm_sal.head(10))

#### Z-Score Standardization on Age

In [ ]:
# Standardize age and print first 5
age_z = (vstable["Age"] - vstable["Age"].mean()) / vstable["Age"].std()
age_z.head(5)

In [ ]:
# Add a new column to the data frame
vstable["Age-Std"] = age_z
vstable.head()

In [ ]:
inc_bins = pd.qcut(vstable.Income, 3)
inc_bins.head(10)

In [ ]:
# Bin the income variable and print first few
inc_bins = pd.qcut(vstable.Income, [0, .33, .66, 1], labels=["low", "mid", "high"])
inc_bins.head(10)

In [ ]:
# Concatenate vstable with two new vars
vs2 = pd.concat([vstable,inc_bins], axis=1)
vs2.head(10)

In [ ]:
# Or add binned income directly to vstable
vstable["inc-bins"] = inc_bins
vstable.head(2)

In [ ]:
# We can also drop columns from the dataframe
vstable.drop(columns=['Age-Std','inc-bins'], inplace=True)
vstable.head()

In [ ]:
# Create a copy of vstable with numeric values only
vs_numeric = vstable[["Age","Income","Rentals","AvgPerVisit"]]
vs_num_std = (vs_numeric - vs_numeric.mean()) / vs_numeric.std()
vs_num_std.head(5)

In [ ]:
#A lambda function is a small anonymous function.
zscore = lambda x: (x - x.mean()) / x.std()
vs_num_std = vs_numeric.apply(zscore)
vs_num_std.head()

In [ ]:
# Instead of separating the numeric attributes, we can condition the standardization function on the data types
zscore = lambda x: ((x - x.mean()) / x.std()) if (x.dtypes==np.float64 or x.dtypes==np.int64) else x
vs_std = vstable.copy()
vs_std.apply(zscore).head()

### Grouping and aggregating data

In [ ]:
# Aggregate data using groupby
vstable.groupby("Gender").mean()

In [ ]:
vstable.groupby("Genre").mean()

In [ ]:
#vstable.groupby("Genre").describe()
vstable.groupby("Genre").describe().T

In [ ]:
# Plotting a histogram
vstable["Income"].plot(kind="hist")
#vstable["Income"].plot(kind="hist", bins=4)

In [ ]:
# Bar plot is different - discrete counts, so we can change the color
vstable["Genre"].value_counts().plot(kind='bar', color=['red', 'blue', 'purple'])

In [ ]:
#vstable["Genre"].value_counts()
#vstable["Genre"].value_counts()
#vstable["Genre"].count()

In [ ]:
# Sometimes it is more useful to look at percentages
temp1 = vstable["Genre"].value_counts()/vstable["Genre"].count()
temp2 = vstable["Gender"].value_counts()/vstable["Gender"].count()

fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(121)
ax1.set_xlabel('Genre')
ax1.set_ylabel('Percentage')
ax1.set_title("Genre Distribution")
temp1.plot(kind='bar', grid = True)

ax1 = fig.add_subplot(122)
ax1.set_xlabel('Gender')
ax1.set_ylabel('Percentage')
ax1.set_title("Gender Distribution")
temp2.plot(kind='bar', grid = True)


In [ ]:
# Scatter plots (look at relationships between two variables)
vstable.plot(x="Income", y="Age", kind="scatter")

In [ ]:
# Use a third variable to color the data points and convey more information
vstable.plot(x="Income", y="Age", kind="scatter", alpha=0.8, s=vstable["Rentals"]*5, c="AvgPerVisit", cmap=plt.get_cmap("jet"), colorbar=True, figsize=(10,7))

In [ ]:
# Using groupby and count for cross-tabulation
vstable.groupby(["Genre","Gender"])["Gender"].count()

In [ ]:
# Or use the crosstab function
gg = pd.crosstab(vstable["Genre"], vstable["Gender"])
gg

In [ ]:
# Figures are easier to read
plt.show(gg.plot(kind="bar"))

In [ ]:
gg["percent_female"] = gg["F"]/(gg["F"]+gg["M"])
gg

In [ ]:
plt.show(gg["percent_female"].plot(kind="bar"))

#### Suppose that we would like to find all "good cutomers", i.e., those with Rentals value of >= 30:

In [ ]:
good_cust = vstable[(vstable.Rentals>=30)]
good_cust


In [ ]:
print ("Good Customers:\n", good_cust.describe())
print ("\n All Customers:\n", vstable.describe())

In [ ]:
filteringDataframe = vstable[(vstable.Rentals>=30) & (vstable.Gender=='F')]
filteringDataframe

#### Creating dummy variables and converting to standard spreadsheet format (all numeric attributes)

In [ ]:
gender_bin = pd.get_dummies(vstable["Gender"], prefix="Gender")
gender_bin.head()

In [ ]:
vs_ssf = pd.get_dummies(vstable)
vs_ssf.head(10)

In [ ]:
#vs_ssf.describe(include="all")
vs_ssf.describe()

In [ ]:
# Min-Max normalization performed on the full numeric data set
vs_norm = (vs_ssf - vs_ssf.min()) / (vs_ssf.max()-vs_ssf.min())
vs_norm.head(10)

In [ ]:
# After converting to all numeric attributes, we can perform correlation analysis on the variable
corr_matrix = vs_ssf.corr()
corr_matrix

In [ ]:
corr_matrix["Rentals"].sort_values(ascending=False)

#### The new table can be written into a file using to_csv method:

In [ ]:
vs_norm.to_csv("../data/video_store_Numeric.csv", float_format="%1.2f")